In [1]:
import pandas as pd
# import sys to get more detailed Python exception info
import sys
# import the connect library for psycopg2
import psycopg2
import psycopg2.extras as extras
# import the error handling libraries for psycopg2
from psycopg2 import OperationalError, errorcodes, errors

In [2]:
ingredient = pd.read_csv('./ingredient.csv')
recipe = pd.read_csv('./recipe.csv')
connections = pd.read_csv('./connections.csv')
unit = pd.read_csv('./unit.csv')

### Specify the connection parameters

In [3]:
conn_params_dic = {
    "host"      : "localhost",
    "database"  : "recipes_2",
    "user"      : "postgres",
    "password"  : "***"
}

### Support Functions

In [4]:
# Define a function that handles and parses psycopg2 exceptions
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()    
    # get the line number when exception occured
    line_n = traceback.tb_lineno    
    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type) 
    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)    
    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")
    
# Define a connect function for PostgreSQL database server
def connect(conn_params_dic):
    conn = None
    try:
        print('Connecting to the PostgreSQL...........')
        conn = psycopg2.connect(**conn_params_dic)
        print("Connection successfully..................")
        
    except OperationalError as err:
        # passing exception to function
        show_psycopg2_exception(err)        
        # set the connection to 'None' in case of error
        conn = None
    return conn

        
# Define function using psycopg2.extras.execute_values() to insert the dataframe.
def execute(conn, datafrm, table):
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(datafrm.columns))
    
    # SQL query to execute
    sql = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, sql, tpls)
        conn.commit()
        print("Data inserted using execute_values() successfully...")
    except (Exception, psycopg2.DatabaseError) as err:
        # pass exception to function
        show_psycopg2_exception(err)
        cursor.close()

### Perform main task

In [5]:
# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True
# Run the execute_many method
execute(conn, ingredient, 'ingredient')
# Close the connection
conn.close()

Connecting to the PostgreSQL...........
Connection successfully..................
Data inserted using execute_values() successfully...


In [6]:
# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True
# Run the execute_many method
execute(conn, recipe, 'recipe')
# Close the connection
conn.close()

Connecting to the PostgreSQL...........
Connection successfully..................
Data inserted using execute_values() successfully...


In [7]:
# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True
# Run the execute_many method
execute(conn, connections, 'connections')
# Close the connection
conn.close()

Connecting to the PostgreSQL...........
Connection successfully..................
Data inserted using execute_values() successfully...


In [8]:
# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True
# Run the execute_many method
execute(conn, unit, 'unit')
# Close the connection
conn.close()

Connecting to the PostgreSQL...........
Connection successfully..................
Data inserted using execute_values() successfully...


### Query the database to check

In [9]:
# Connect to the database
conn = connect(conn_params_dic)
cursor = conn.cursor()

# Execute query
sql = "SELECT * FROM recipe"
cursor.execute(sql)

# Fetch all the records
tuples = cursor.fetchall()

# list of columns
cols = list(recipe.columns)

recipe_df_read = pd.DataFrame(tuples,columns=cols)

display(recipe_df_read.head(3))

# Close the cursor
cursor.close()

# Close the conn
conn.close()

Connecting to the PostgreSQL...........
Connection successfully..................


,id_recipe,cuisine,meal_type,name_recipe,portions,calories,protein,fat,carbo,text_recipe,vegeterian
0,0,Русская кухня,Завтраки,Сырники из творога,2,938.0,42.0,61.0,56.0,Положите весь творог в кастрюльку и разомните ...,False
1,1,Русская кухня,Выпечка и десерты,Классическая шарлотка,12,217.0,5.0,5.0,38.0,Разогреть духовку. Отделить белки от желтков. ...,False
2,2,Европейская кухня,Основные блюда,Свинина с карамелизованной морковью,4,551.0,39.0,32.0,28.0,Свинину зачистить и обсушить. Зубчики чеснока ...,False


### Query the database to check

In [11]:
# Connect to the database
conn = connect(conn_params_dic)
cursor = conn.cursor()

# Execute query
sql = "SELECT * FROM ingredient"
cursor.execute(sql)

# Fetch all the records
tuples = cursor.fetchall()

# list of columns
cols = list(ingredient.columns)

ingredient_read = pd.DataFrame(tuples,columns=cols)

display(ingredient_read.head())

# Close the cursor
cursor.close()

# Close the conn
conn.close()

Connecting to the PostgreSQL...........
Connection successfully..................


,id_ingr,name_ingr
0,0,Творог
1,1,Куриное яйцо
2,2,Пшеничная мука
3,3,Сахар
4,4,Подсолнечное масло
